# Упражнение 7.2

В этой главе показано, как выразить ДПФ и обратное ДПФ произведениями матриц. Время выполнения этих операций пропорционально N^2, где N - длина массива сигнала. Во многих случаях этого достаточно, но есть алгоритм и побыстрее - быстрое преобразование Фурье (БПФ); время его работы пропорционально Nlog N. Ключ к БПФ - это лемма Дэниелсона-Ланцоша (Danielson- Lanczos).

Эта лемма предлагает рекурсивный алгоритм для ДПФ:

1. Дан массив сигнала у. Разделим его на четные элементы е и нечетные элементы о.
2. Вычислим DFT е и о, делая рекурсивные вызовы.
3. Вычислим DFT(y) для каждого значения n, используя лемму Дэниелсона-Ланцоша.

В простейшем случае эту рекурсию надо продолжать, пока длина у не дойдет до 1. Тогда DFT(y) = у. А если длина у достаточно мала, можно вычислить его ДПФ перемножением матриц, используя заранее вычисленные матрицы.

Давайте начнем с исследования функции FFT из библиотеки. Затем мы применим быстрое преобразование Фурье (БПФ) к небольшому реальному сигналу.

In [ ]:
import numpy as np
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Давайте определим функцию для вычисления дискретного преобразования Фурье (DFT).

In [ ]:
PI2 = 2 * np.pi

def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(1j * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

In [ ]:
hs2 = dft(ys)
np.sum(np.abs(hs - hs2))

5.864775846765962e-16

Мы создали функцию dft, которая вычисляет дискретное преобразование Фурье для входного сигнала ys, используя массивы времени ts, частот freqs, аргументы args, матрицу M, и возвращая амплитуды преобразования, после чего мы сравнили результаты с другими значениями.

Затем разработаем функцию, которая разбивает массив на две равные части и применяет быстрое преобразование Фурье к каждой из них.

In [ ]:
def fft_norec(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

In [ ]:
hs3 = fft_norec(ys)
np.sum(np.abs(hs - hs3))

0.0

Полученные результаты соответствуют ожиданиям.

Далее, мы заменим функцию np.fft.fft на рекурсивные вызовы и проверим новую функцию для вычисления БПФ.

In [ ]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys

    He = fft(ys[::2])
    Ho = fft(ys[1::2])

    ns = np.arange(N)
    W = np.exp(-1j * PI2 * ns / N)

    return np.tile(He, 2) + W * np.tile(Ho, 2)

И проверим.

In [ ]:
hs4 = fft(ys)
np.sum(np.abs(hs - hs4))

1.6653345369377348e-16

Функции FFT (Быстрое Преобразование Фурье) и DFT (Дискретное Преобразование Фурье) применяются для перевода сигналов из временной области в частотную. Обе функции осуществляют одинаковое преобразование, однако FFT выполняет это операцию значительно быстрее благодаря использованию алгоритма, основанного на разложениях в ряд Фурье. Время работы FFT составляет O(n log(n)), в то время как для DFT оно равно O(n^2). Таким образом, FFT является более эффективным алгоритмом для вычисления преобразования Фурье, особенно для обработки больших объемов данных.